In [1]:
import ctypes
from datetime import datetime

#path='/home/lorenzo/git/lc_pulse_avalanche/pyMEPSA/mepsa.so' # LB
path='C:/Users/lisaf/Desktop/GitHub/lc_pulse_avalanche/pyMEPSA/mepsa.so'      # AF

start_program = datetime.now()

#------------------------------------------------------------------------------#
# A. Create C library in Python 
#------------------------------------------------------------------------------#
# We load the C library (mepsa.c) via ctypes.CDLL(<libpath>), where <libpath>
# is a relative pointer to the shared library file (mepsa.so). To create the
# shared library just execute the Makefile inside the 'pyMEPSA' folder.
mepsa_lib = ctypes.CDLL(path)

#------------------------------------------------------------------------------#
# B. Specify function signatures
#------------------------------------------------------------------------------#
# All functions defined within the library mepsa.c have now become methods of
# the 'mepsa_lib' object defined above.
# Python is a dynamically typed language, whereas C is statically typed language. 
# This discrepancy between Python and C is important because we generally want
# to pass information between Python and C. ctypes will try to infer the 
# datatype, but it can mess up the inference of a C datatype and we are left
# with garbage values being passed in either direction. Catastrophic. Failure.
# To address this issue, ctypes allows us to explicitly specify the function
# signatures (i.e., the datatypes associated with the input and output values
# of a given function).  
# The main function of MEPSA takes as input 4 (actually 5) strings:
#   (- name of the program);
#    - input time series (3 columns: time, rate, error on rate);
#    - excess patterns' file;
#    - maximum rebin factor to be searched;
#    - name of the file to be saved with the results;
# Choose the C function in the library to import in Python (the main function
# in our case):
peak_find = mepsa_lib.main
# Types of the _inputs_ of the C function imported (in our case, the inputs of
# the main function):
peak_find.argtypes = ctypes.c_int, ctypes.POINTER(ctypes.c_char_p)
# Type of the _output_ of the C function imported (in our case, the return of
# the main function):
peak_find.restype  = ctypes.c_int 

#------------------------------------------------------------------------------#
# C. Invoke function
#------------------------------------------------------------------------------#
test_grb = b'./test/input_lc.dat'
exp_file = b'./excess_pattern_MEPSA_v0.dat'
reb      = b'16'
grb_name = 'GRB12345'
out_file = (grb_name + '.dat').encode('ascii')
argv     = (ctypes.c_char_p * 5) (b'pyMEPSA', test_grb, exp_file, reb, out_file)
#print(out_file)
start_peak_find = datetime.now()
peak_find(len(argv), argv)
end_time = datetime.now()
#------------------------------------------------------------------------------#

print("Time to load the libraries: ", (start_peak_find - start_program)  )
print("Time to execute MEPSA:      ", (end_time        - start_peak_find))

OSError: [WinError 193] %1 non è un'applicazione di Win32 valida